# Imports

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.functional import one_hot

import numpy as np
import pandas as pd
from collections import Counter
from random import choice, random, randint
import os
import re
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from unicodedata import normalize

nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package rslp to
[nltk_data]     D:\Users\maraujo\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     D:\Users\maraujo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     D:\Users\maraujo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Preparando dados

## Listando arquivos

In [2]:
texts_dir = './artigos'
texts_list = [f'{texts_dir}/{file}' for file in  os.listdir(texts_dir)]

todas_sentencas = []
for text in texts_list:
    arquivo = open(text, 'r', encoding='utf-8')
    sentencas = arquivo.readlines()
    arquivo.close()

    todas_sentencas.append(sentencas)

todas_sentencas = np.array(todas_sentencas, dtype=np.object)
todas_sentencas = np.hstack(todas_sentencas)

df = pd.DataFrame(todas_sentencas, columns=['sentencas'])
df.head()

,sentencas
0,'Queremos viralizar': a realidade por trás das...
1,Recém-casados em algumas partes da Índia estão...
2,"Nayana, de 23 anos, é enfermeira. Abhijith, de..."
3,"“Queríamos que viralizasse”, diz Nayana. Ela a..."
4,Nayana diz que acompanha as fotos de noivos in...


## Limpando dados

In [3]:
stemmer = nltk.stem.RSLPStemmer() # Objeto para "stemização" em português

def remover_acentos(frase):
    return normalize('NFKD', frase).encode('ASCII', 'ignore').decode('ASCII')

RETIRAR_STOP_WORDS = False
ESTEMIZAR = False

if (RETIRAR_STOP_WORDS):
    stopwords = [remover_acentos(sw) for sw in  nltk.corpus.stopwords.words('portuguese')]
else:
    stopwords = []

def normalizar_frase(frase):
    frase = str(frase).lower()
    frase = remover_acentos(frase)
    frase = re.sub(r'\w*[0-9]+\w*', '', frase)
    frase = re.sub(r'[\[\]º!"#$%&\'()*+,-./:;<=>?@^_`~]+', '', frase) # remove caracteres especiais
    
    tokens = [stemmer.stem(tk) if ESTEMIZAR else tk for tk in word_tokenize(frase) if tk not in stopwords] # removendo stopwords e estemizando

    return tokens

df['tokens'] = df['sentencas'].apply(normalizar_frase)
df.sample(3)

,sentencas,tokens
89989,Essa traição foi um golpe duro.\n,"[essa, traicao, foi, um, golpe, duro]"
57042,A maioria dos casos de tuberculose em 2019 oco...,"[a, maioria, dos, casos, de, tuberculose, em, ..."
54540,Agora temos que continuar espalhando a mensage...,"[agora, temos, que, continuar, espalhando, a, ..."


## Criando Vocabulário e Dicionário

In [4]:
counter = Counter()
for tokens in df['tokens'].values:
    counter.update(tokens)

FREQUENCIA_MINIMA = 2
words = [tk for tk,freq in list(counter.items()) if freq >= FREQUENCIA_MINIMA]
words2idx = {w:n for n,w in enumerate(words)}
idx2words = {n:w for n,w in enumerate(words)}

## Refatorando novamente a lista
- Neste passo são removidos os tokens que baixa frequência além de transformá-los em seus índices

In [5]:
# Limpando as sentenças novamente agora só com as palavras oficiais
sentencas_final = []
for sentenca in tqdm(df['tokens'].values, ncols=100):
    vetor = [words2idx[w] for w in sentenca if w in words]
    sentencas_final.append(vetor)

100%|████| 136040/136040 [12:45<00:00, 177.82it/s]


## Preparando o dataset
- Agora criamos o dataset (de acordo com seus índices) utilizando o algoritmo [CBOW Context Bag of Words](https://en.wikipedia.org/wiki/Word2vec#CBOW_and_skip_grams)

In [20]:
class custom_dataset(Dataset):

    def __init__(self, n_window, lista_sentencas):
        self.lista_sentencas = lista_sentencas
        self.tensor_x, self.tensor_y = [], []
        for sentenca in tqdm(self.lista_sentencas, ncols=100):
            for ind_tk in range(len(sentenca)-2*n_window):
                vetorx = sentenca[ind_tk:ind_tk+2*n_window+1]
                vetory = vetorx.pop(n_window)
                self.tensor_x.append(vetorx)
                self.tensor_y.append(vetory)
        
        self.tensor_x = torch.tensor(self.tensor_x, dtype=torch.long)
        self.tensor_y = torch.tensor(self.tensor_y)

    def __getitem__(self, index):
        return self.tensor_x[index], self.tensor_y[index]
    
    def __len__(self):
        return len(self.tensor_x)

dataset = custom_dataset(n_window=2, lista_sentencas=sentencas_final)

100%|████████████████████████████████████████████████████| 136040/136040 [00:06<00:00, 21361.63it/s]


# Criação do modelo
- Vamos criar o modelo de rede neural bem simples, apenas utilizando camada linear e treinar de acordo com nosso dataset

In [98]:
class Modelo_CBOW(nn.Module):

    def __init__(self, embedding_dim, vocab_size):
        super(Modelo_CBOW, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, max_norm=1)
        self.linear = nn.Sequential(
            nn.ReLU(),
            nn.Linear(in_features=embedding_dim, out_features=vocab_size),
            nn.Softmax(dim=-1)
        )
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.mean(axis=1)
        return self.linear(x)

model = Modelo_CBOW(embedding_dim=32, vocab_size=len(words))
model

Modelo_CBOW(
  (embedding): Embedding(53278, 32, max_norm=1)
  (linear): Sequential(
    (0): ReLU()
    (1): Linear(in_features=32, out_features=53278, bias=True)
    (2): Softmax(dim=-1)
  )
)

# Treinamento da rede neural

In [99]:
EMBEDDING_DIM = 32
N_EPOCHS = 5
BATCH_SIZE = 16
LEARNING_RATE = 1e-3

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model = Modelo_CBOW(embedding_dim=EMBEDDING_DIM, vocab_size=len(words))
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(DEVICE)
model.train()

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(N_EPOCHS):
    loss_epoch = 0
    for x, y_real in tqdm(dataloader, ncols=50):
        x, y_real = x.to(DEVICE), y_real.to(DEVICE)
        
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y_real)
        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
    
    print (f'[{epoch}]: {round(loss_epoch,3)}')

  0%|      | 256/231720 [00:20<5:08:19, 12.51it/s]


KeyboardInterrupt: 